In [1]:
# This is the template for the submission. You can develop your algorithm in a regular Python script and copy the code here for submission.

# TEAM NAME ON KAGGLE
# CJY

# GROUP NUMBER
# group_00

# TEAM MEMBERS (E-MAIL, LEGI, KAGGLE USERNAME):
# "cachtari@student.ethz.ch", "19-827-898", "cachtari" 
# "yabboud@ethz.ch", "19-808-864", "youmnabb"
# "jrihani@student.ethz.ch", "20-810-511", "jeanclauderihani"

In [2]:
import os
import random
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path
from scipy.stats import skew, kurtosis
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
from scipy.stats import entropy

In [3]:
# Get the path for all test traces
dir_traces = '/kaggle/input/24-exercise2/data/test'
filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f))]
filenames.sort()

In [4]:
#------ Location ------
X_loc = np.load('/kaggle/input/features/group00_features_0.npy', allow_pickle=True)
labels_loc = np.load('/kaggle/input/features/group00_features_2.npy', allow_pickle=True)
X_train_loc, X_val_loc, y_train_loc, y_val_loc = train_test_split(X_loc, labels_loc, test_size=0.2, random_state=42)

X_path = np.load('/kaggle/input/features/group00_features_1.npy', allow_pickle=True)
labels_path = np.load('/kaggle/input/features/group00_features_3.npy', allow_pickle=True)
X_train_path, X_val_path, y_train_path, y_val_path = train_test_split(X_path, labels_path, test_size=0.2, random_state=42)

X_act = np.load('/kaggle/input/features/group00_features_4.npy', allow_pickle=True)
labels_act = np.load('/kaggle/input/features/group00_features_5.npy', allow_pickle=True)
X_train_act, X_val_act, y_train_act, y_val_act = train_test_split(X_act, labels_act, test_size=0.2, random_state=42)

In [5]:
scaler_loc = StandardScaler()
X_train_loc = scaler_loc.fit_transform(X_train_loc)
X_val_loc = scaler_loc.transform(X_val_loc)


clf_loc = xgb.XGBClassifier()
clf_loc.fit(X_train_loc, y_train_loc)

train_preds = clf_loc.predict(X_train_loc)
val_preds = clf_loc.predict(X_val_loc)

accuracy = accuracy_score(y_train_loc, train_preds)
balanced_accuracy = balanced_accuracy_score(y_train_loc, train_preds)

print("Training accuracy:", accuracy)
print("Training balanced accuracy:", balanced_accuracy)

print()

accuracy = accuracy_score(y_val_loc, val_preds)
balanced_accuracy = balanced_accuracy_score(y_val_loc, val_preds)

print("Validation accuracy:", accuracy)
print("Validation balanced accuracy:", balanced_accuracy)

Training accuracy: 1.0
Training balanced accuracy: 1.0

Validation accuracy: 0.95
Validation balanced accuracy: 0.9487907925407925


In [6]:
scaler_path = StandardScaler()
X_train_path = scaler_path.fit_transform(X_train_path)
X_val_path = scaler_path.transform(X_val_path)

clf_path = xgb.XGBClassifier()
clf_path.fit(X_train_path, y_train_path)

train_preds = clf_path.predict(X_train_path)
val_preds = clf_path.predict(X_val_path)

accuracy = accuracy_score(y_train_path, train_preds)
balanced_accuracy = balanced_accuracy_score(y_train_path, train_preds)

print("Training accuracy:", accuracy)
print("Training balanced accuracy:", balanced_accuracy)

print()

accuracy = accuracy_score(y_val_path, val_preds)
balanced_accuracy = balanced_accuracy_score(y_val_path, val_preds)

print("Validation accuracy:", accuracy)
print("Validation balanced accuracy:", balanced_accuracy)

Training accuracy: 1.0
Training balanced accuracy: 1.0

Validation accuracy: 0.8
Validation balanced accuracy: 0.7881350434910805


In [7]:
scaler_act = StandardScaler()
X_train_act = scaler_act.fit_transform(X_train_act)
X_val_act = scaler_act.transform(X_val_act)


clf_act = xgb.XGBClassifier()
clf_act.fit(X_train_act, y_train_act)

train_preds = clf_act.predict(X_train_act)
val_preds = clf_act.predict(X_val_act)

accuracy = accuracy_score(y_train_act, train_preds)
balanced_accuracy = balanced_accuracy_score(y_train_act, train_preds)

print("Training accuracy:", accuracy)
print("Training balanced accuracy:", balanced_accuracy)

print()

accuracy = accuracy_score(y_val_act, val_preds)
balanced_accuracy = balanced_accuracy_score(y_val_act, val_preds)

print("Validation accuracy:", accuracy)
print("Validation balanced accuracy:", balanced_accuracy)

Training accuracy: 1.0
Training balanced accuracy: 1.0

Validation accuracy: 0.9411764705882353
Validation balanced accuracy: 0.9374669312169311


In [8]:
import numpy as np
from scipy.stats import skew, kurtosis, iqr

def extract_features_location(signals, num_windows=10):
    
    features = []
    
    for signal in signals:
        num_samples = len(signal)
        window_size = num_samples // num_windows
        step_size = window_size // 2

        # Extract features in windows with 50% overlap
        for i in range(0, num_samples - window_size + 1, step_size):
            window = signal[i:i+window_size]

            # Normalization
            window = (window - np.mean(window)) / np.std(window)

            # Statistical features
            features.append(np.mean(window))
            features.append(np.var(window))
            features.append(skew(window))
            features.append(kurtosis(window))
            features.append(np.max(window) - np.min(window))  # Difference of max and min values
            features.append(iqr(window))  # Interquartile range

            # Frequency domain features (using FFT)
            window *= np.hamming(len(window))  # Applying Hamming window
            fft_result = np.fft.fft(window)
            features.extend([np.abs(fft_result).mean(), np.abs(fft_result).std()])

            # Time-domain features
            energy = np.sum(np.square(window))
            features.append(energy)

    return features


def extract_features_path_old(signal, num_windows=10):
    features = []
    num_samples = len(signal)
    window_size = num_samples // num_windows
    step_size = window_size // 2

    # Counter to track the number of windows processed
    window_count = 0

    # Extract features in windows with 50% overlap
    for i in range(0, num_samples - window_size + 1, step_size):
        if window_count == num_windows:
            break  # Stop after processing the desired number of windows
        
        window = signal[i:i+window_size]
        
        # Statistical features
        features.append(np.mean(window))
        features.append(np.var(window))
        features.append(np.max(window) - np.min(window))

        # Frequency domain features (using FFT)
        fft_result = np.fft.fft(window)
        features.extend([np.abs(fft_result).mean(), np.abs(fft_result).std()])

        # Time-domain features
        energy = np.sum(np.square(window))
        features.append(energy)

        # Slope feature
        slope = (window[-1] - window[0]) / window_size 
        features.append(slope)

        window_count += 1

    return features

def low_pass_filter(data, cutoff=3, order=1, sample_rate=200):
    
    nyquist_freq = 0.5 * sample_rate
    normal_cutoff = cutoff / nyquist_freq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = signal.filtfilt(b, a, data)
    
    return filtered_data

def moving_average(signal, window_size):
    
    weights = np.repeat(1.0, window_size) / window_size
    smoothed_signal = np.convolve(signal, weights, 'valid')
    
    return smoothed_signal

def frequency_domain_entropy(signal, sampling_rate):
    # Compute the FFT
    fft_result = np.fft.fft(signal)
    
    # Exclude the DC component (first element)
    fft_magnitudes = np.abs(fft_result[1:])
    
    # Normalize magnitudes
    fft_magnitudes /= np.sum(fft_magnitudes)
    
    # Calculate entropy
    entropy_value = entropy(fft_magnitudes)
    
    return entropy_value

import numpy as np
from scipy.stats import entropy

def frequency_domain_entropy(signal, sampling_rate):
    # Compute the FFT
    fft_result = np.fft.fft(signal)
    
    # Exclude the DC component (first element)
    fft_magnitudes = np.abs(fft_result[1:])
    
    # Normalize magnitudes
    fft_magnitudes /= np.sum(fft_magnitudes)
    
    # Calculate entropy
    entropy_value = entropy(fft_magnitudes)
    
    return entropy_value

def extract_features_act(signals, window_size=1000, sampling_rate=200):
    
    features = []

    num_samples = len(signals[0])
    step_size = window_size // 2
    
    # Iterate over windows
    for i in range(0, num_samples - window_size + 1, step_size):
            
        window = np.array([signal[i:i+window_size] for signal in signals])

        # Statistical features
        window_mean = np.mean(window, axis=1)
        window_std = np.std(window, axis=1)
        features.extend(window_mean)
        features.extend(window_std)

        # Energy
        window_energy = np.sum(np.square(np.abs(np.fft.fft(window))), axis=1) / window_size
        features.extend(window_energy)

        # Correlation
        covxy = np.cov(window[0], window[1])[0, 1]
        covyz = np.cov(window[1], window[2])[0, 1]
        covxz = np.cov(window[0], window[2])[0, 1]
        
        stdx = np.std(window[0])
        stdy = np.std(window[1])
        stdz = np.std(window[2])
        
        features.append(covxy / (stdx*stdy))
        features.append(covyz / (stdy*stdz))
        features.append(covxz / (stdx*stdz))
        
        # Interquartile Range (IQR)
        window_iqr = np.percentile(window, 75, axis=1) - np.percentile(window, 25, axis=1)
        features.extend(window_iqr)
        
        # Max-Min Range
        window_max = np.max(window, axis=1)
        window_min = np.min(window, axis=1)
        window_range = window_max - window_min
        features.extend(window_range)
        
        # Frequency-domain entropy
        for signal in window:
            entropy_value = frequency_domain_entropy(signal, sampling_rate)
            features.append(entropy_value)
                
    return features

from scipy.signal import butter, filtfilt

def bandpass_filter(data, cutoff=0.5, order=1, fs=200):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data


In [9]:
def find_repeated_integers(lst):
    repeated_integers = []
    current_integer = None
    repetition_count = 0

    for num in lst:
        if num == current_integer:
            repetition_count += 1
        else:
            if repetition_count >= 3:
                repeated_integers.append(current_integer)
            current_integer = num
            repetition_count = 1

    # Check for the last sequence
    if repetition_count >= 3:
        repeated_integers.append(current_integer)

    return repeated_integers


In [10]:
def band_filter(data, lc=0.3, order=1, rate=200):
    b, a = butter(order, lc, btype='high', fs=rate)
    return filtfilt(b, a, data)

import numpy as np
from scipy.signal import find_peaks

def find_peaks_dynamic_threshold(data, window_size=200, threshold_factor=3, smoothing_window=30, post_processing=True):
    # Apply smoothing to the data
    smoothed_data = np.convolve(data, np.ones(smoothing_window)/smoothing_window, mode='same')
    
    # Calculate dynamic threshold
    threshold = np.std(smoothed_data[:window_size]) * threshold_factor
    
    # Find peaks using dynamic thresholding
    peaks, _ = find_peaks(smoothed_data, height=threshold, distance=window_size)
    
    # Perform post-processing if enabled
    if post_processing:
        # Example of merging nearby peaks
        peaks = merge_nearby_peaks(peaks, window_size)
    
    return peaks


def merge_nearby_peaks(peaks, min_distance):
    merged_peaks = []
    i = 0
    while i < len(peaks):
        peak = peaks[i]
        while i + 1 < len(peaks) and peaks[i + 1] - peak < min_distance:
            i += 1
        merged_peaks.append(peak)
        i += 1
    return merged_peaks


def remove_steps(peaks, preds):
    removed_peaks = []
    for i in range(len(preds)):
        if (preds[i] == 1) or (preds[i] == 2):
            window_start = i * 3000
            window_end = (i + 1) * 3000
            for peak in peaks:
                if window_start <= peak < window_end:
                    removed_peaks.append(peak)
    return removed_peaks

def normalize(data):
    return (data - np.mean(data)) / np.std(data)


In [11]:
# Get the path for all test traces
dir_traces = '/kaggle/input/24-exercise2/data/test'
filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f))]
filenames.sort()

# Loop through all filenames to process recordings
submission = []
for filename in filenames:
    recording = Recording(filename)
    
    # Assumes filename format ends with a three-digit ID before ".pkl"
    match = re.search(r'(\d{3})\.pkl$', filename)
    if match:
        id = int(match.group(1))
    else:
        raise ValueError(f'Filename {filename} does not match expected format')
    
    features = np.empty(7, dtype=object)

    features[0] = np.array(recording.data['ax'].to_dict()['values'])
    features[1] = np.array(recording.data['ay'].to_dict()['values'])
    features[2] = np.array(recording.data['az'].to_dict()['values'])
    features[3] = np.array(recording.data['gx'].to_dict()['values'])
    features[4] = np.array(recording.data['gy'].to_dict()['values'])
    features[5] = np.array(recording.data['gz'].to_dict()['values'])
    features[6] = np.array(recording.data['altitude'].to_dict()['values'])

    features_act_windowed_test = []
    labels_act_windowed_test = []

    for j in range(int(len(features[0]) // 3000)):
        features_act_temp = np.empty(6, dtype=object)  # Create a new array for each iteration
        for k in range(6):
            features_act_temp[k] = features[k][j * 3000:(j + 1) * 3000]

        features_act_windowed_test.append(features_act_temp)

    features_act_windowed_test = np.array(features_act_windowed_test)
    
    for i in range(len(features_act_windowed_test)):
        features_act_windowed_test[i, 0] = bandpass_filter(features_act_windowed_test[i, 0])
        features_act_windowed_test[i, 1] = bandpass_filter(features_act_windowed_test[i, 1])
        features_act_windowed_test[i, 2] = bandpass_filter(features_act_windowed_test[i, 2])
        features_act_windowed_test[i, 3] = bandpass_filter(features_act_windowed_test[i, 3])
        features_act_windowed_test[i, 4] = bandpass_filter(features_act_windowed_test[i, 4])
        features_act_windowed_test[i, 5] = bandpass_filter(features_act_windowed_test[i, 5])

    features_act_windowed_test = np.array([extract_features_act(sample, window_size=1000) for sample in features_act_windowed_test])

    test_scaled = scaler_act.transform(features_act_windowed_test)
    pred = clf_act.predict(test_scaled)
    preds = list(set(find_repeated_integers(pred)))
    
    features[0] = low_pass_filter(features[0])
    features[1] = low_pass_filter(features[1])
    features[2] = low_pass_filter(features[2])
    features[3] = low_pass_filter(features[3])
    features[4] = low_pass_filter(features[4])
    features[5] = low_pass_filter(features[5])
    
    features[6] = moving_average(features[6], 100)
    
    loc = np.array(extract_features_location(features[0:6], num_windows=10))
    path = np.array(extract_features_path_old(features[6], num_windows=10))
    
    loc = scaler_loc.transform(loc.reshape(1, -1))
    path = scaler_path.transform(path.reshape(1, -1))
    
    step_feat = np.sqrt(features[0]**2 + features[1]**2 + features[2]**2)
    step_feat_filtered = band_filter(step_feat)
    step_feat_filtered = normalize(step_feat_filtered)
    peaks = find_peaks_dynamic_threshold(step_feat_filtered, window_size=50, threshold_factor=3, smoothing_window=30, post_processing=True)
    peaks = remove_steps(peaks, pred)

    path_idx = clf_path.predict(path)[0] # Integer, path in {0, 1, 2, 3, 4}
    watch_loc = clf_loc.predict(loc)[0]  # Integer, 0: left wrist, 1: belt, 2: right ankle
    standing = False  # Boolean, True if participant was standing still throughout the recording
    walking = False  # Boolean, True if participant was walking throughout the recording
    running = False  # Boolean, True if participant was running throughout the recording
    cycling = False  # Boolean, True if participant was cycling throughout the recording
    step_count = len(peaks)  # Integer, number of steps, must be provided for each recording

    for activity in preds:
        if activity == 0:
            standing = True
        elif activity == 1:
            walking = True
        elif activity == 2:
            running = True
        elif activity == 3:
            cycling = True
    
    predictions = {
        'Id': id, 
        'watch_loc': watch_loc, 
        'path_idx': path_idx,
        'standing': standing,
        'walking': walking,
        'running': running,
        'cycling': cycling,
        'step_count': step_count
        }

    submission.append(predictions)

In [12]:
# Write the predicted values into a .csv file to then upload the .csv file to Kaggle
# When cross-checking the .csv file on your computer, we recommend using a text editor and NOT excel so that the results are displayed correctly
# IMPORTANT: Do NOT change the name of the columns of the .csv file ("Id", "watch_loc", "path_idx", "standing", "walking", "running", "cycling", "step_count")
submission_df = pd.DataFrame(submission, columns=['Id', 'watch_loc', 'path_idx', 'standing', 'walking', 'running', 'cycling', 'step_count'])
submission_df.to_csv('/kaggle/working/submission.csv', index=False)